In [2]:
# common_functions.py
from pathlib import Path
from neuralhydrology.utils.config import Config
from helpers import get_predictor_list, get_static_attr_list

def create_date_range_train(fold: int):
    if fold == 0:
        return "01/01/1981", "31/12/1997"
    elif fold == 1:
        return "01/01/2000", "31/12/2014"

def create_date_range_test(fold: int):
    if fold == 1:
        return "01/01/1981", "31/12/1997"
    elif fold == 0:
        return "01/01/2000", "31/12/2014"

def generate_configs(config_file: Path, static_attributes: list, dynamic_perdictor_options: list,output_dir: Path=None, date_ranges: dict = None, basin_files:dict = None, prefix='test'):
    output_dir.mkdir(exist_ok=True, parents=True)

    for static_attrs in static_attributes:
        for predictors in dynamic_perdictor_options:
            for fold in [0, 1]:
                cfg = Config(config_file)

                cfg.update_config({
                    'dynamic_inputs': get_predictor_list(predictors),
                    'static_attributes': get_static_attr_list(static_attrs),
                    'experiment_name': f'{prefix}_{fold}_{predictors}_{static_attrs}',
                })
                if basin_files:
                    train_basin_file = basin_files['train'][fold] 
                    validation_basin_file = basin_files['validation'][fold]
                    test_basin_file = basin_files['test'][fold]
                                        
                    cfg.update_config({
                        'train_basin_file': train_basin_file,
                        'test_basin_file': test_basin_file,
                        'validation_basin_file': validation_basin_file,
                    })

                if date_ranges:
                    train_start_date, train_end_date = date_ranges['train'][fold]
                    validation_start_date, validation_end_date = date_ranges['validation'][fold]
                    test_start_date, test_end_date = date_ranges['test'][fold]

                    cfg.update_config({                
                        'train_start_date': train_start_date,
                        'train_end_date': train_end_date,
                        'validation_start_date': validation_start_date,
                        'validation_end_date': validation_end_date,
                        'test_start_date': test_start_date,
                        'test_end_date': test_end_date,
                    })          

                # Start training
                cfg.dump_config(output_dir, cfg.experiment_name + '.yml')



In [6]:
# common configs
base_path = Path('/Users/sho108/Desktop/r/work/sho108/neuralhydrology_workflow/configs2')
config_file = Path('config.yml')


static_attributes = [
    '',
    'Are',  # Catchment Area
    'Pme',  # Precipitation Mean
    'Pet',  # PET Mean
    'Run',  # Runoff Ratio
    'Slo',  # Slope FDC
    'Str',  # Stream Elasticity
    'Bas',  # Baseflow Index
    'Hpf',  # High Precipitation Frequency
    'Hpd',  # High Precipitation Duration
    'Lpf',  # Low Precipitation Frequency
    'Lpd',  # Low Precipitation Duration
    'Mea',  # Mean Slope Percentage
    'Ksa',  # Saturation Hydraulic Conductivity (Ksat)
    'Ari',  # Aridity Index
    'Pse',  # Precipitation Seasonality Index
]



dynamic_predictors = ['Pra',
              'PraEts',
              'Prs',
              'PrsEts',
              'PrsEas',
              ]

In [7]:
# Temporal
# This cell contains the main script for generating temporal split configurations in neural hydrology experiments.

date_ranges = {
    'train': {0: ("01/01/1981", "31/12/1997"), 1: ("01/01/2000", "31/12/2014")},
    'validation': {0: ("01/01/2000", "31/12/2014"), 1: ("01/01/1981", "31/12/1997")},
    'test': {0: ("01/01/2000", "31/12/2014"), 1: ("01/01/1981", "31/12/1997")},
}

basin_files = {'train' : {fold: f'basins/basins.txt' for fold in [0,1]},
'validation' : {fold: f'basins/basins.txt' for fold in [0,1]},
'test' : {fold: f'basins/basins.txt' for fold in [0,1]}}

generate_configs(config_file,
                [''],
                dynamic_predictors,
                output_dir = base_path / 'compare_dynamic_predicotors/temporal_split/twofold',
                date_ranges=date_ranges,
                basin_files=basin_files,
                prefix='temporal_twofold')

generate_configs(config_file,
                static_attributes,
                ['PraEts'],
                output_dir = base_path / 'compare_static_attr_predicotors/temporal_split/twofold',
                date_ranges=date_ranges,
                basin_files=basin_files,
                prefix='temporal_twofold')




In [8]:
# Spatial
# This cell contains the main script for generating spatial split configurations in neural hydrology experiments.

basin_files = {'train' : {fold: f'basins/twofoldsplit/fold_{fold}_train.txt' for fold in [0,1]},
'validation' : {fold: f'basins/twofoldsplit/fold_{fold}_test.txt' for fold in [0,1]},
'test' : {fold: f'basins/twofoldsplit/fold_{fold}_test.txt' for fold in [0,1]}}


generate_configs(config_file,
                [''],
                dynamic_predictors,
                output_dir = base_path / 'compare_dynamic_predicotors/spatial_split/twofold',
                basin_files=basin_files,
                prefix='spatial_twofold')

generate_configs(config_file,
                static_attributes,
                ['PraEts'],
                output_dir = base_path / 'compare_static_attr_predicotors/spatial_split/twofold',
                basin_files=basin_files,
                prefix='spatial_twofold')

